In [1]:
using Revise, Pkg
Pkg.activate("/home/louise/MSA/BpAlignGpu.jl")
using BpAlignGpu

  Activating project at `~/MSA/BpAlignGpu.jl`
┌ Info: Precompiling BpAlignGpu [5a3eb610-29b2-4cbe-8ba2-ea97f65fa95d]
└ @ Base loading.jl:1423


In [2]:
#(q, N, L) = (21, 161, 67) 
(q, N, L) = (21, 5, 5) 

(21, 5, 5)

In [3]:
function convert_T3_xnTOy(tmp)
    SCE_ar = fill(0.0, 2N+4, L)
    for i=1:L
        for n=1:N+2
            SCE_ar[n, i] = tmp[n,1,i]
            SCE_ar[N+2+n, i] = tmp[n,2,i]
        end
    end
    return SCE_ar
end
function convert_T5_xnTOy(tmp)
    SCE_ar = fill(0.0, 2N+4, 2N+4, L)
    for i=1:L
        for n1=1:N+2
            for n2=1:N+2
                SCE_ar[n1, n2, i] = tmp[n1, 1, n2, 1, i]
                SCE_ar[n1, N+2+n2, i] = tmp[n1, 1, n2, 2, i]
                SCE_ar[N+2+n1, n2, i] = tmp[n1, 2, n2, 1, i]
                SCE_ar[N+2+n1, N+2+n2, i] = tmp[n1, 2, n2, 2, i]
            end
        end
    end
    return SCE_ar
end
function convert_T6_xnTOy(tmp)
    SCE_ar = fill(0.0, 2N+4, 2N+4, L, L)
    for i=1:L
        for j=1:L
            for n1=1:N+2
                for n2=1:N+2
                    SCE_ar[n1, n2, i, j] = tmp[n1, 1, n2, 1, i, j]
                    SCE_ar[n1, N+2+n2, i, j] = tmp[n1, 1, n2, 2, i, j]
                    SCE_ar[N+2+n1, n2, i, j] = tmp[n1, 2, n2, 1, i, j]
                    SCE_ar[N+2+n1, N+2+n2, i, j] = tmp[n1, 2, n2, 2, i, j]
                end
            end
        end
    end
    return SCE_ar
end

convert_T6_xnTOy (generic function with 1 method)

In [4]:
T = Float32

Float32

In [5]:
using Random
header = "myseq"
myseq = randstring('A':'Z', N)
ctype=Symbol("amino")
seq = Seq(header, myseq, ctype)

myseq
IJAIG


In [6]:
muint = 1.5
muext = 1.2
lambda_o = ones(L)
lambda_e = ones(L)
H = rand(q,L)
J = rand(q,q,L,L)
J = J .+ permutedims(J, (2,1,4,3));

In [7]:
pm = ParamModel{T}(N, L, q, muint, muext, lambda_o, lambda_e, H, J)

ParamModel{Float32}[L=5 N=5 q=21]


In [8]:
damp=T(0.0)
tol=T(1e-5)
tolnorm=T(1e-5)
tmax=10
upscheme=:sequential # :random or :sequential
lr=:sce  # :sce or :mf 
beta=T(1.0)
verbose=true
pa = ParamAlgo(damp, tol, tolnorm, tmax, upscheme, lr, beta, verbose)

ParamAlgo{Float32}
-------------
damp=0.0
tol=1.0e-5
tolnorm=1.0e-5
tmax=10
upscheme=sequential
lr=sce
beta=1.0
verbose=true
-------------


In [9]:
bpm = BPMessages(seq, pm)

┌ Warning: The NVIDIA driver on this system only supports up to CUDA 11.0.0.
│ For performance reasons, it is recommended to upgrade to a driver that supports CUDA 11.2 or higher.
└ @ CUDA /home/louise/.julia/packages/CUDA/bki2w/src/initialization.jl:70


BPMessages{Float32}[L=5 N=5 ongpu=true]


In [10]:
bpb = BPBeliefs(N, L)

BPBeliefs{Float32}[L=5 N=5 ongpu=true]

In [11]:
lrf = LongRangeFields(N, L)

LongRangeFields{Float32}[L=5 N=5 ongpu=true]

In [12]:
af = AllFields(bpm, bpb, lrf)

AllFields{Float32}[L=5 N=5 ongpu=true]

# Checks

In [13]:
tmp_F = Array(af.bpm.F)
SCE_F = convert_T3_xnTOy(tmp_F);

In [14]:
tmp_hF = Array(af.bpm.hF)
SCE_hF = convert_T3_xnTOy(tmp_hF);

In [15]:
tmp_hB = Array(af.bpm.hB)
SCE_hB = convert_T3_xnTOy(tmp_hB);

In [16]:
tmp_B = Array(af.bpm.B)
SCE_B = convert_T3_xnTOy(tmp_B);

In [17]:
tmp_g = Array(af.lrf.g)
SCE_g = convert_T5_xnTOy(tmp_g);

In [18]:
tmp_f = Array(af.lrf.f)
SCE_f = convert_T3_xnTOy(tmp_f);

In [19]:
tmp_Jseq = Array(af.bpm.Jseq)
elts_Jseq = convert_T6_xnTOy(tmp_Jseq);

In [20]:
tmp_Hseq = Array(af.bpm.Hseq)
elts_Hseq = convert_T3_xnTOy(tmp_Hseq);

In [21]:
include("../src/SCE_update_messages.jl")
include("../src/SCE_update_shortrange_beliefs.jl")
include("../src/SCE_update_longrange.jl")

SCE_update_g! (generic function with 1 method)

## check BP updates

In [22]:
BpAlignGpu.update_hF!(af, pm, pa)
#BpAlignGpu.update_hB!(af, pm, pa)
#BpAlignGpu.update_F!(af, pm, pa)
#BpAlignGpu.update_B!(af, pm, pa)

In [23]:
i=2
af.bpm.hF[:,:,i]

7×2 CUDA.CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 0.00398591  0.0
 0.104514    0.00390063
 0.0995933   0.1085
 0.0269677   0.0859979
 0.0966738   0.0787347
 0.0896295   0.0732652
 0.228238    0.0

In [24]:
#SCE_update_F!(SCE_F, i, damp, beta, tolnorm, SCE_hF, SCE_f, muext, muint, N, elts_Hseq)
#SCE_update_B!(SCE_B, i, damp, beta, tolnorm, SCE_hB, SCE_f, muext, muint, N, L, elts_Hseq)
SCE_update_hF!(SCE_hF, i, SCE_F, damp, beta, tolnorm, SCE_g, pm.lambda_o, pm.lambda_e, N, elts_Jseq)
#SCE_update_hB!(SCE_hB, i, damp, beta, tolnorm, SCE_B, SCE_g, pm.lambda_o, pm.lambda_e, N, elts_Jseq)

In [25]:
SCE_hF[:,]

14×5 Matrix{Float64}:
 0.0301421   0.00398591  0.078527   0.0825693  0.192846
 0.108448    0.104514    0.0903562  0.0490269  0.252727
 0.113946    0.0995933   0.0504421  0.0836217  0.0659749
 0.203007    0.0269677   0.138968   0.0717867  0.0170056
 0.145842    0.0966738   0.0669898  0.0961742  0.00556181
 0.00917046  0.0896295   0.0774503  0.0766164  0.000803516
 0.0350826   0.228238    0.0810313  0.0291353  0.104511
 0.0         0.0         0.0        0.0        0.0
 0.0576262   0.00390063  0.107538   0.0955321  0.0683588
 0.112199    0.1085      0.0377997  0.0802408  0.163179
 0.00240475  0.0859979   0.0514351  0.103943   0.00801437
 0.142196    0.0787347   0.0890287  0.11732    0.0336996
 0.0399365   0.0732652   0.130434   0.114034   0.0873188
 0.0         0.0         0.0        0.0        0.0

## check update beliefs

In [29]:
#tmp_bel = Array(af.bpb.beliefs)
#SCE_bel = convert_T3_xnTOy(tmp_bel);

In [30]:
#BpAlignGpu.update_beliefs!(af, pm)

In [31]:
#i=1
#SCE_update_beliefs!(SCE_bel, i, beta, tolnorm, SCE_hF, SCE_hB, SCE_f, muint, muext, N, L, elts_Hseq)

In [32]:
#af.bpb.beliefs[:,:,i]

In [33]:
#SCE_bel[:,i]

## check update joint proba

In [34]:
#tmp_joint = Array(af.bpb.joint_chain)
#SCE_joint = convert_T5_xnTOy(tmp_joint);

In [35]:
#sum(SCE_joint)

In [36]:
BpAlignGpu.update_jointchain!(af, pm)

In [37]:
#sum(af.bpb.joint_chain, dims=(1,2,3,4))[1,1,1,1,:]

In [38]:
#i=1
#af.bpb.joint_chain[:,1,:,1,i]

In [39]:
#SCE_update_joint_chain!(SCE_joint, i, beta, tolnorm, SCE_F, SCE_B, SCE_g, lambda_o, lambda_e, N, elts_Jseq);

In [40]:
#rx1 = 1:N+2
#rx2 = N+3:2N+4
#SCE_joint[rx1,rx1, i]

In [41]:
#sum(SCE_joint, dims=(1,2))[1,1,:]

## check update conditional on the chain

In [42]:
BpAlignGpu.update_conditional_chain!(af, pa)

In [43]:
#tmp_joint = Array(af.bpb.joint_chain)
#SCE_joint = convert_T5_xnTOy(tmp_joint);

In [44]:
#cond_chain = fill(0.0, 2N+4, 2N+4, L, L)
#for i=1:L-1
#    SCE_update_conditional_chain!(i, cond_chain, SCE_joint, tolnorm, N)
#end

In [45]:
#af.bpb.conditional[:,2,:,1,1,2]

In [46]:
#cond_chain[rx2, rx1, 1, 2]

In [47]:
#sum(cond_chain[:,:,4,3], dims=1)

In [48]:
#sum(af.bpb.conditional[:,:,:,:,4,3], dims=(1,2))[1,1,:,:]

## check update conditional off the chain

In [49]:
#cond_lr = copy(cond_chain);
#pointer(cond_chain), pointer(cond_lr)

In [50]:
#SCE_update_conditional_lr!(cond_lr)

In [51]:
BpAlignGpu.update_conditional_all!(af, pm);

In [52]:
#cond_lr[rx1, rx2, 4, 1]

In [53]:
#af.bpb.conditional[:,1,:,2,4,1]

In [54]:
#sum(af.bpb.conditional[:,:,:,:,4,1], dims=(1,2))[1,1,:,:]

In [55]:
#sum(cond_lr[:,:,4,1], dims=1)

## check update f

In [58]:
tmp_cond= Array(af.bpb.conditional)
SCE_cond = convert_T6_xnTOy(tmp_cond);

In [59]:
tmp_f = Array(af.lrf.f)
SCE_f = convert_T3_xnTOy(tmp_f);

In [60]:
af.lrf.f[:,:,2]

7×2 CUDA.CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0

In [61]:
BpAlignGpu.update_f!(af)

In [63]:
using LinearAlgebra
SCE_update_f!(SCE_f, SCE_cond, elts_Jseq, N)

In [64]:
i=3
af.lrf.f[:,:,i]

7×2 CUDA.CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 -4.24892   0.0
 -4.0828   -4.27414
 -3.85625  -4.70392
 -4.40407  -3.8992
 -4.08144  -4.17385
 -1.88375  -3.41813
 -4.19178   0.0

In [65]:
SCE_f[:,i]

14-element Vector{Float64}:
 -4.248915610112336
 -4.082803403479909
 -3.856248602006823
 -4.404074380157429
 -4.081436507314701
 -1.8837479043673235
 -4.191776273157981
  0.0
 -4.274139669206367
 -4.703921061363093
 -3.899199102371751
 -4.17384718815496
 -3.418132984940926
  0.0

## check update g

In [66]:
tmp_cond= Array(af.bpb.conditional)
SCE_cond = convert_T6_xnTOy(tmp_cond);

In [67]:
tmp_g = Array(af.lrf.g)
SCE_g = convert_T5_xnTOy(tmp_g);

In [68]:
af.lrf.g[:,1,:,2,2]

7×7 CUDA.CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0

In [69]:
BpAlignGpu.update_g!(af)

In [72]:
SCE_update_g!(SCE_g, SCE_cond, elts_Jseq, N)

In [87]:
i=1
af.lrf.g[:,1,:,1,i]

7×7 CUDA.CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 3.45272  3.45279  3.15216  3.11431  2.71212  2.16102  3.51013
 3.45272  3.45279  3.15216  3.11431  2.71212  2.16102  3.51013
 3.45272  3.45279  3.15216  3.11431  2.71212  2.16102  3.51013
 3.45272  3.45279  3.15216  3.11431  2.71212  2.16102  3.51013
 3.45272  3.45279  3.15216  3.11431  2.71212  2.16102  3.51013
 3.45272  3.45279  3.15216  3.11431  2.71212  2.16102  3.51013
 3.45272  3.45279  3.15216  3.11431  2.71212  2.16102  3.51013

In [88]:
rx1 = 1:N+2
rx2 = N+3:2N+4
SCE_g[rx1,rx1,i]

7×7 Matrix{Float64}:
 3.45272  3.45279  3.15216  3.11431  2.71212  2.16102  3.51013
 3.45272  3.45279  3.15216  3.11431  2.71212  2.16102  3.51013
 3.45272  3.45279  3.15216  3.11431  2.71212  2.16102  3.51013
 3.45272  3.45279  3.15216  3.11431  2.71212  2.16102  3.51013
 3.45272  3.45279  3.15216  3.11431  2.71212  2.16102  3.51013
 3.45272  3.45279  3.15216  3.11431  2.71212  2.16102  3.51013
 3.45272  3.45279  3.15216  3.11431  2.71212  2.16102  3.51013